In [106]:
import numpy as np
import torch
import torch.nn.functional as F

In [3]:
with open("../Makemore/names.txt") as f:
    names = f.readlines()

In [6]:
names = [name.strip() for name in names]

In [21]:
char_set = sorted(list({c for name in names for c in name}))
char_set.insert(0, '.')
len(char_set)

27

In [23]:
stoi = {char: i for i, char in enumerate(char_set)}
itos = {i: char for char, i in stoi.items()}

In [95]:
block_size = 3
X, Y = [], []
for name in names[:5]:
    name = '.' * block_size + name + '.'
    for i in range(len(name) - block_size):
        context = name[i: i+block_size]
        X.append([stoi[char] for char in context])
        Y.append(stoi[name[i+block_size]])

X = torch.tensor(X)
Y = torch.tensor(Y)

In [96]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [115]:
C =  torch.randn((27, 2))

In [116]:
W1 = torch.randn((6, 100))
B1 = torch.randn(100)
W2 = torch.randn((100, 27))
B2 = torch.randn(27)
parameters = [C, W1, B1, W2 ,B2]

In [126]:
for param in parameters:
    param.requires_grad = True

In [246]:
out = torch.tanh(torch.matmul(C[X].view(-1, 6), W1) + B1)
out.shape

torch.Size([32, 100])

In [247]:
out = torch.matmul(out, W2) + B2
out.shape

torch.Size([32, 27])

In [248]:
counts = out.exp()
probs = torch.div(counts, counts.sum(dim=1, keepdims=True))
loss = -probs[torch.arange(X.shape[0]), Y].log().mean()
loss

tensor(0.7142, grad_fn=<NegBackward0>)

In [249]:
loss.backward()
for param in parameters:
    param.data += -0.1 * param.grad
    param.grad = None